# DF

In [ ]:
%store -r df_one_hot

# PACKAGES

In [ ]:
# package that we need
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,accuracy_score, precision_score, recall_score,f1_score
from sklearn.model_selection import KFold

# essential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#correlation
import seaborn as sns

#! pip install pydot

# FEATURE AND TARGET

In [33]:
numerical_features = [
    'spotify_track_duration_minute', 'danceability', 'energy', 'loudness',
    'mode', 'speechiness', 'acousticness',
    'spotify_track_popularity', 'instrumentalness', 'liveness']

genre_features = [col for col in df_one_hot.columns if col.startswith('genre_')]

# UNION OF THE TWO 
total_features = numerical_features + genre_features

target='valence'

# PLOT

interesting plots for visualize the output of our model

In [ ]:
def plot_for_model(history):

    #plot loss
    plt.figure()
    plt.plot(history.history['loss'],color='blue',label='loss')
    plt.plot(history.history['val_loss'],color='red',label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss function through epoch')
    plt.legend()
    plt.grid()
    plt.show()

    #plot mean
    plt.figure()
    plt.plot(history.history['mae'],color='blue',label='mae')
    plt.plot(history.history['val_mae'],color='red',label='val_mae')
    plt.xlabel('Epoch')
    plt.ylabel('mae')
    plt.title('Mae function through epoch')
    plt.grid()
    plt.legend()
    plt.show()   

# SPLIT DATA

In [ ]:
def return_train_test_for_cross(train_index, test_index, df):
    #extract the feature and target
    x = df[total_features]
    y = df[target]

    #normalization
    scaler = StandardScaler()
    x[numerical_features] = scaler.fit_transform(x[numerical_features])
    #display(x)

    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #display(x_train)

    return x_train,x_test,y_train,y_test

# MODEL

In [ ]:
# function to create the model
def create_model_user(shape_x_train): #number of columns -> shape
    
    
    '''model_user_i = Sequential([
        Dense(32, input_dim=shape_x_train, activation='relu', name='first_layer'),
        Dense(16, activation='relu', name='secod_layer'),
        Dense(1, activation='sigmoid', name='final_layer')
    ])'''

    model_user_i = Sequential([
        
        Dense(32,input_dim=shape_x_train,activation='relu',name='first_layer',kernel_regularizer=l2(0.01)),
        Dropout(0.3),
        Dense(16,activation='relu',name='secod_layer',kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        #Dense(1,activation='linear',name='final_layer')
        Dense(1,activation='sigmoid',name='final_layer') 
    ])

    #compile the model
    model_user_i.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

    #summary
    #display(model_user_i.summary())

    #return  the model
    return model_user_i
    

# FIT MODEL

In [ ]:
#function to train the model
def fit_model(model, x_train, y_train, x_test, y_test):

    history_model = model.fit(
        x_train,y_train,
        validation_data = [x_test, y_test],
        epochs=100,
        batch_size=32,
        #verbose=1
    )

    return history_model

# RECOMANDATION FUNCTION + CONFUSION MATRIX

In [ ]:
def reco_function_conf_matrix(y_test, y_hat, threshold):
    
    #initialize dataframe    
    data = {'Original Test': y_test,
        'Prediction Test': y_hat.flatten(),
        }
    df_prediction = pd.DataFrame(data = data)
    df_prediction['Original over Thrashold'] = df_prediction['Original Test'] > threshold
    df_prediction['Predicted over Thrashold'] = df_prediction['Prediction Test'] > threshold
    display(df_prediction)

    #plot confusion matrix of reccomandation
    cm = confusion_matrix(df_prediction['Original over Thrashold'], df_prediction['Predicted over Thrashold'])

    # METRIX FOR EVALUATE RECCOMANDATION 
    # Extract TP, TN, FP, FN
    TN, FP, FN, TP = cm.ravel()

    # Calculate metrics manually
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    plt.figure()
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['True', 'False'], yticklabels=['True', 'False'])
    plt.title('Confiusion matrix')
    plt.xlabel('predicted')
    plt.ylabel('actual')
    plt.show()

    # Store metrics in a DataFrame
    df_metrics_cm = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score'],
        'Value': [accuracy, precision, recall, f1]
    })

    display(df_metrics_cm.round(2))

    return df_prediction,df_metrics_cm

# MAIN 

In [ ]:
dizio_model_user = {} #in this dizio we store the model for each user
dizio_accuracy = {} #accuracy of the model

# DICTIONARY TO STORE SONG RECOMMENDATION PER USER
reccomandations_songs_per_user={}

#threshold
threshold = 0.7

#value k for k-fold
k = 5

#loop creation user and model
for i in range(0, len(df_one_hot['User'].unique())): #i indica lo user   

    print('---------------------------------------------------------')
    print(f'USER {i}')

    #select user
    mask_user = df_one_hot['User'] == i
    #display(mask_user)
    user_i_df = df_one_hot[mask_user]
    #display(user_i_df) 

    #cross-validation k-fold
    kf = KFold(n_splits = k, shuffle= True, random_state= 42)
    score_fold = []

    #loop on the k-th folder
    loop = 0
    for train_index, test_index in kf.split(user_i_df):
        
        print('---------------------------------------------------------')
        print(f'K {loop} FOLD') 
        loop = loop + 1

        #return train and test
        x_train,x_test,y_train,y_test = return_train_test_for_cross(train_index, test_index, user_i_df) #da modificare

        #creation model for the specific user
        model_user_i = create_model_user(x_train.shape[1])
        
        #fit the model with the different x, y train (for different user)
        history_model_user_i = fit_model(model_user_i, x_train, y_train, x_test, y_test)
        
        #prediction
        #y_hat = model_user_i.predict(x_test)

        #plot the metrics of the model
        #plot_for_model(history_model_user_i)

        #accuracy
        test_loss, test_mae = model_user_i.evaluate(x_test,y_test,verbose=1)
        #print('MSE: ', test_loss)
        #print('MAE:', test_mae)

        score_fold.append(test_mae)

        #reccomandation
        #reccomandation_df = reco_function_conf_matrix(y_test, y_hat, threshold)

    #save the model in the dictionary
    #add also the raccomandations
    dizio_model_user[f"User{i}"] = {#'model_user_i' : model_user_i,
                                    #'history_model_user_i': history_model_user_i,
                                    #'x_train': x_train,
                                    #'x_test': x_test,
                                    #'y_train': y_train,
                                    #'y_test': y_test,
                                    #'y_hat': y_hat,
                                    #'recommendations:': reccomandation_df
                                    'score_fold': score_fold
                                    } 
    #save accuracy 
    dizio_accuracy[f'User{i}'] = {
        #'MAE' : mean_absolute_error(y_test, y_hat),
        #'MSE' : mean_squared_error(y_test, y_hat)
        'Mean MAE': np.mean(score_fold)
    }

---------------------------------------------------------
USER 0
---------------------------------------------------------
K 0 FOLD


C:\Users\elped\AppData\Local\Temp\ipykernel_8780\2667823296.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[numerical_features] = scaler.fit_transform(x[numerical_features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.4505 - mae: 0.2062 - val_loss: 0.1863 - val_mae: 0.1675
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1523 - mae: 0.1744 - val_loss: 0.0742 - val_mae: 0.1572
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0674 - mae: 0.1621 - val_loss: 0.0459 - val_mae: 0.1503
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0478 - mae: 0.1586 - val_loss: 0.0396 - val_mae: 0.1464
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0429 - mae: 0.1546 - val_loss: 0.0381 - val_mae: 0.1462
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0414 - mae: 0.1533 - val_loss: 0.0377 - val_mae: 0.1461
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0404 - mae: 0.1531 - val_loss: 0.0369 - val_mae: 0.1442
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0388 - mae: 0.1486 - val_loss: 0.0369 - val_mae: 0.1442
Epoch 9/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

C:\Users\elped\AppData\Local\Temp\ipykernel_8780\2667823296.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[numerical_features] = scaler.fit_transform(x[numerical_features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.4638 - mae: 0.2101 - val_loss: 0.1979 - val_mae: 0.1759
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1605 - mae: 0.1767 - val_loss: 0.0808 - val_mae: 0.1656
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0688 - mae: 0.1604 - val_loss: 0.0500 - val_mae: 0.1570
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0487 - mae: 0.1606 - val_loss: 0.0429 - val_mae: 0.1574
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0421 - mae: 0.1551 - val_loss: 0.0409 - val_mae: 0.1510
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0394 - mae: 0.1494 - val_loss: 0.0397 - val_mae: 0.1502
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0400 - mae: 0.1520 - val_loss: 0.0391 - val_mae: 0.1487
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0400 - mae: 0.1510 - val_loss: 0.0388 - val_mae: 0.1499
Epoch 9/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/

...

63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.5759 - mae: 0.2756 - val_loss: 0.3742 - val_mae: 0.1925
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3410 - mae: 0.2001 - val_loss: 0.2369 - val_mae: 0.1792
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2188 - mae: 0.1906 - val_loss: 0.1543 - val_mae: 0.1727
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1425 - mae: 0.1763 - val_loss: 0.1057 - val_mae: 0.1665
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0988 - mae: 0.1688 - val_loss: 0.0774 - val_mae: 0.1606
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0751 - mae: 0.1661 - val_loss: 0.0610 - val_mae: 0.1561
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0622 - mae: 0.1651 - val_loss: 0.0516 - val_mae: 0.1518
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0511 - mae: 0.1535 - val_loss: 0.0463 - val_mae: 0.1496
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0485 - mae: 0.

C:\Users\elped\AppData\Local\Temp\ipykernel_8780\2667823296.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[numerical_features] = scaler.fit_transform(x[numerical_features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.5053 - mae: 0.2271 - val_loss: 0.3429 - val_mae: 0.1929
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3107 - mae: 0.1960 - val_loss: 0.2178 - val_mae: 0.1843
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1978 - mae: 0.1842 - val_loss: 0.1426 - val_mae: 0.1788
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1298 - mae: 0.1773 - val_loss: 0.0990 - val_mae: 0.1743
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0921 - mae: 0.1728 - val_loss: 0.0736 - val_mae: 0.1683
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0696 - mae: 0.1677 - val_loss: 0.0597 - val_mae: 0.1657
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0573 - mae: 0.1640 - val_loss: 0.0518 - val_mae: 0.1613
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0501 - mae: 0.1592 - val_loss: 0.0473 - val_mae: 0.1588
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0476 - mae: 0

C:\Users\elped\AppData\Local\Temp\ipykernel_8780\2667823296.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[numerical_features] = scaler.fit_transform(x[numerical_features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.5396 - mae: 0.2142 - val_loss: 0.3686 - val_mae: 0.1788
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3367 - mae: 0.1856 - val_loss: 0.2322 - val_mae: 0.1696
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2148 - mae: 0.1764 - val_loss: 0.1498 - val_mae: 0.1647
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1375 - mae: 0.1640 - val_loss: 0.1009 - val_mae: 0.1611
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0959 - mae: 0.1636 - val_loss: 0.0725 - val_mae: 0.1575
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0711 - mae: 0.1604 - val_loss: 0.0565 - val_mae: 0.1537
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0561 - mae: 0.1529 - val_loss: 0.0477 - val_mae: 0.1504
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0490 - mae: 0.1533 - val_loss: 0.0429 - val_mae: 0.1478
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0434 - mae: 0.1

# DISPLAY DICTIONARY ACCURAY

In [40]:
display(dizio_accuracy)

{'User0': {'Mean MAE': 0.13768224120140077},
 'User1': {'Mean MAE': 0.13333794772624968},
 'User2': {'Mean MAE': 0.09675337672233582},
 'User3': {'Mean MAE': 0.13251740336418152},
 'User4': {'Mean MAE': 0.13890121281147003}}